# Data Understanding

- [Data Collection](#data_collection)
- [Data Exploration](#data_exploration)

---

## Data Collection <a name="data_collection"></a>

Um später aussagekräftige Modelle erstellen zu können, müssen im ersten Schritt die dafür notwendigen Daten gesammelt und ein Datensatz erstellt werden. Dazu werden Beiträge auf der Plattform Twitter gesammelt und in einer Liste gespeichert.

#### 1. Daten sammeln

Um ausschließlich relevante und die Branche betreffende Twitter-Beiträge zu sammeln werden folgende Suchparameter übergeben:
- hashtags: #technology, #tech, #innovation
- language: en
- period: until:2023-03-31 since:2022-10-01
- limit: 1_000_000

In [2]:
%%script false
from src.data.nitter_scraper_standalone_v2 import *
from datetime import datetime

# set search parameters
query = '%28%23technology+OR+%23tech+OR+%23innovation%29+lang%3Aen&e-nativeretweets=on'
since = datetime(2022, 10, 1)
until = datetime(2023, 3, 31)
limit = 1_000_000


# collect twitter posts
scrape(q=query, since=since, until=until, limit=limit)

Couldn't find program: 'false'


#### 2. Datensatz anzeigen

Nachdem die Twitter-Beiträge gesammelt wurden, werden diese in ein Dataframe transformiert und angezeigt.

In [8]:
from src.data.nitter_scraper_standalone_v2 import Tweet, TweetScraper
import pandas as pd

# load tweet objects
list_of_tweets = TweetScraper.load_collected_tweets(path='../data/raw/twitter_tweets_raw.pkl')

# transform into a dataframe
dict_of_tweets =  [{"url": tweet.url, "date": tweet.date, "rawContent": tweet.rawContent} for tweet in list_of_tweets]
df = pd.DataFrame(dict_of_tweets)

In [9]:
from IPython.display import display

# set up jupyter to be able to display the entire dataframe (specifically all columns)
pd.options.display.max_columns = None

df.head(4)

,url,date,rawContent
0,https://twitter.com/_Bob_S/status/164159110508...,"Mar 30, 2023 · 11:59 PM UTC",Govt IT security isnt 'a nice thing to do': it...
1,https://twitter.com/WYSIWYGVentures/status/164...,"Mar 30, 2023 · 11:59 PM UTC",ISC West 2023: Cyberattackers Are Targeting Ph...
2,https://twitter.com/HackerAran7/status/1641591...,"Mar 30, 2023 · 11:59 PM UTC",What’s the hack. #stem #science #stemeducation...
3,https://twitter.com/bytefeedai/status/16415909...,"Mar 30, 2023 · 11:59 PM UTC",BuzzFeed Is Using AI To Write SEO-Bait Travel ...


---

## Data Exploration <a name="data_exploration"></a>

Nun gilt es den gesammelten Datensatz zu beschreiben, zu untersuchen und abschließend zu bewerten. Die Untersuchung dient dem Zweck die Daten hinsichtlich ihrer Qualität zu bewerten und Maßnahmen für das Data Cleaning zu beschließen.

#### 1. Überblick

In [10]:
df.dtypes

url           object
date          object
rawContent    object
dtype: object

#### 2. Überprüfe Anforderungen an den Datensatz

Um die Qualität der Daten bewerten zu können, muss zuerst sichergestellt werden, ob die Daten auch den zuvor aufgestellen Anforderungen entsprechen. Nachfolgend werden daher folgende Überprüfungen vorgenommen:
- Prüfe ob alle Beiträge im angegebenen Zeitraum erstellt wurden
- Prüfe ob alle Beiträge Textdaten enthalten
- Prüfe ob Duplikate existieren 
- Prüfe ob alle Beiträge in englischer Sprache sind

In [11]:
# check whether all posts were created within the selected period
# expected result: True
import datetime

_ = df.copy()
_['date'] = _['date'].apply(lambda x: datetime.datetime.strptime(x, "%b %d, %Y · %I:%M %p %Z"))

_.query('date < "2022-10-01" or date > "2023-03-31"').empty

True

In [12]:
# check if entries have no text
# expected result: False

df['rawContent'].isnull().any()

False

In [17]:
# check for duplicates
# expected result: False

df['rawContent'].duplicated().any()

True

In [19]:
# check if all texts are in English
# expected result: True
from langdetect import detect

_ = df.copy()

# define function to identify language of tweet
def detect_language(text):
    try:
        lang = detect(text)
    except:
        lang = None
    return lang

_['lang'] = _['rawContent'].apply(detect_language)
_['lang'].eq('en').all()

False

---